# Gereksinim Kontrolleri

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print("GPU'ya bağlı olmamanız nedeniyle işleme devam edilemez.")
else:
  print(gpu_info)

Mon May 27 18:46:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Gerekli kütüphanelerin kurulumu ve projeye dahil edilmesi

In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install xformers[cuda]
!pip install wandb

import pandas as pd
import torch

from datasets import Dataset, DatasetDict
from unsloth import FastLanguageModel

from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8q5e2j_j/unsloth_541fdf1104f84d28952788136721ab6c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8q5e2j_j/unsloth_541fdf1104f84d28952788136721ab6c
  Resolved https://github.com/unslothai/unsloth.git to commit b0781339f035c72b3028d846eb2261e8115cd375
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
  Created wheel for uns

# LLM Modelinin Yüklenmesi ve Başlangıç Ayarları


In [3]:
max_seq_length = 2048 # Modele girecek metin girdisinin maksimum uzunluğunu ifade eder. 2048 token olarak tanımladık.
dtype = None # Unslothda kullanacağımız veri türünü belirleyebiliriz. None olarak bırakırsak varsayılan float32 olarak işlem yapar.
load_in_4bit = True # Model ağırlığını 4 bit hassasiyetle yüklemek isteyip istemediğimizi belirtiriz. Bu işlem bellek kullanımını önemli ölçüde azaltır fakat modelin doğruluğunu biraz etkileyebilir.

# FastLanguageModel.from_pretrained() : Önceden eğtilmiş olan bir LLM modelini yüklüyoruz.
# Model_name alanında 8 milyar parametreli llama-3 modelinin 4 bit ağırlıklı versiyonunu yükleyeceğimizi belirtiyoruz.
# max_seq_length, dtype, load_in_4bit alanlarına ise yukarıda tanımladığımız değişken verilerini atıyoruz.
# Kısacası aşağıdaki kod, belirtilen model adını, maksimum dizi uzunluğunu, veri türünü ve 4 bit yükleme seçeneğini kullanarak FashLanguageModel ->
# -> sınıfından nesne oluşturur. Bu nesne modeli ve metin verilerini tokenlara dönüştürmek için kullanılacak bir tokenizer içerir.
# tokenizer ve token ne demek? Bu yazılacak.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit", #8 milyar parametreli gemma-7b-bnb modelinin 4 bit ağırlıklı versiyonunu yükleyeceğimizi belirtiyoruz.
    max_seq_length = max_seq_length, #modelin işleyebileceği en uzun dizinin (kelime veya token dizisi) uzunluğunu belirtir. Dizinin uzunluğu, modelin bir seferde işleyebileceği maksimum karakter veya kelime sayısını ifade eder.
    dtype = dtype, # Modelin kullanacağı veri tipini belirtir.
    load_in_4bit = load_in_4bit # Modelin 4-bit ağırlıkla yüklenip yüklenmeyeceğini belirtir. Bu, modelin hafıza kullanımını azaltarak daha az yer kaplamasını ve bazı durumlarda daha hızlı çalışmasını sağlar.
)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

# LLM Modelinin PEFT Ayarlarının Yapılandırılması


In [4]:
# PEFT : Positional Encoding Fine-Tuning
# Bu teknik önceden eğitilmiş bir dil modelinin konumsal kodlamasını bir veri seti için özelleştirmeyi sağlar. Daha iyi sonuçlar elde etmek için modelin ağırlıklarını ayarlama işlemini de yapar. Konumsal kodlama, bir cümledeki kelimelerin sırasını modelin anlamasına yardımcı olan bir tekniktir.
# LoRA : Low-Rank Adaptation
# LoRA Gelişmiş bir ince ayar yöntemidir. Hesaplama maliyetini düşürür ve modelin aşırı öğrenmesini önlemeye de yardımcı olur.

# Unsloth kütüphanesinin PEFT ve LoRA özelliklerini kullanarak önceden eğitilmiş bir LLM modelini daha verimli ve göreve özel hale getirmeyi sağlar. Bu teknikler, modelin boyutunu küçültür ve bellek kullanımını azaltır. Böylece performansı artırır.

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,

    #target_modules dizisi, PEFT ve LoRA ile özelleştirilmek istenen modülleri liste olarak tutar. Bu modüller modelin dikkat mekanizmasında ve konumsal kodlamada kullanılacaktır.

    # q_proj: Sorgu projeksiyonu: Diğer tokenlarla olan ilişkilerin değerlendirilmesi için kullanılır.
    # k_proj: Anahtar projeksiyonu: Anahtarlar, sorguların hangi bilgilere odaklanacağını belirlemek için kullanılır.
    # v_proj: Değer projeksiyonu: Girdi verisinin "değer" vektörlerine dönüştürülmesini sağlar.
    # o_proj: Çıkış projeksiyonu: Vektörlerin birleştirilmesi ve dönüştürülmesi işlemlerini kapsar.
    # gate_proj: Kapı projeksiyonu: Bir nevi bilgi akışını kontrol eden bir kapı görevi görür. Hangi bilgilerin ileriye aktarılacağını veya dışlanacağını belirler.
    # up_proj: Yükseltme projeksiyonu: Düşük boyutlu girdileri daha yüksek boyutlu bir uzaya taşıyarak modelin daha fazla bilgiyi işlemesini sağlar.
    # down_proj: Düşürme projeksiyonu: Yüksek boyutlu girdileri daha düşük boyutlu bir uzaya indirgeyerek hesaplama maliyetini azaltır.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    # lora_alpha: Daha yüksek alfa değeri, daha güçlü doğrusallaştırma ve potansiyel olarak daha iyi performans anlamına gelir.
    lora_alpha = 16,

    # lora_dropout: LoRA için dropout oranını belirler. Dropout, aşırı öğrenmeyi önlemek için rastgele nöronları devre dışı bırakır. 0 değeri optimize edilmiş performans için önerilir.
    lora_dropout = 0,

    # bias: "none" değeri, optimize edilmiş performans için önerilir.
    bias = "none",

    # use_gradient_checkpointing: Bellek verimliliği için gradyon kontrol noktasını etkinleştirir. Bellek kullanımını azaltmak için unsloth veya True değerini kullanabiliriz.
    use_gradient_checkpointing = "unsloth",

    # random_state: Yeniden üretilebilirlik için rastgele bir durum belirler. Aynı rastgele durumları kullanarak işlemleri tekrarlamayı sağlayabiliriz.
    random_state = 3407,

    # use_rslora: Rütbe sabitli LoRA'yı etkinleştirir veya devre dışı bırakır. - Modelin ağırlık matrislerini değiştirmek yerine bu ağırlıklara ek dönüşüm matrisleri eklenmesi ile ilgilidir.
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Veri Setinin Yüklenmesi ve Veri Setinin Talimat Şablonuna Entegrasyonu


In [5]:
#Input: Örnek veriyorum mesela instruction'a : Sıradaki örüntüyü tamamla diyip, input'a 1,2,4,8,16 diyebiliriz.
# instruction konuyu ifade eder.
# output için ise cevap veya çözümü veya konunun açıklamasını ifade eder.

def formattingPrompts(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        text = nixous_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# Belirli bir görevi tanımlayan talimatlarla, bu talimatları tamamlayacak girdileri ve bunlara uygun yanıtları eşleştirmek için kullanılır. Bu şablon, yapay zeka modelinin, verilen talimatı anlamasını ve bu talimata uygun olarak verilen girdiye dayanarak doğru bir yanıt üretmesini sağlamak üzere tasarlanmıştır.
nixous_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Bir cümlenin veya girdi dizisinin sona erdiğini belirten bir işaretçidir.

from google.colab import drive
drive.mount('/content/drive')
excel_data = pd.read_excel("/content/drive/MyDrive/new_train.xlsx")
dataset = Dataset.from_pandas(excel_data)
dataset = dataset.map(formattingPrompts, batched=True)

Mounted at /content/drive


Map:   0%|          | 0/797739 [00:00<?, ? examples/s]

# Veri Setinin Analizi


In [6]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 797739
})


# Dil Modelinin Eğitim Ayarları ve Fine-Tune işleminin Gerçekleştirilmesi


In [7]:

# Bu kodun ana görevi, belirtilen parametreler ve yapılandırmalar doğrultusunda bir dil modelinin
# eğitim sürecini başlatmak ve yönetmektir. Eğitim sırasında, model verilen veri seti üzerinde eğitilir
# ve belirtilen ayarlar (örneğin, öğrenme hızı, adım sayısı) doğrultusunda optimize edilir.
# Bu süreç, modelin belirli bir doğal dil işleme görevinde daha iyi performans göstermesi için gereklidir.
# Modelin eğitimi, veri setinden öğrenilen özellikler ve bağlamlar ile geliştirilir, böylece model daha etkili ve doğru tahminler yapabilir.
# Eğitim sonuçları, ileriye dönük uygulamalar veya daha detaylı analizler için outputs dizinine kaydedilir.
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,

    args = TrainingArguments(
        report_to="wandb", # Eğitim istatistiklerinin raporlanacağı hedefleri belirtir.
        per_device_train_batch_size = 2, # Her bir cihazda (GPU/CPU) kullanılacak eğitim örnek sayısını belirtir.
        gradient_accumulation_steps = 4, # Gradyan toplama adımlarını belirtir; bu, daha büyük etkili toplu iş boyutları kullanabilmek için küçük toplu işler üzerinde gradyanların toplanmasını sağlar.
        warmup_steps = 5, # Eğitim başlangıcında öğrenme hızını kademeli olarak artırmak için kullanılan adımların sayısını belirtir.
        #num_train_epochs = 2,
        max_steps = 2000, # Eğitimin maksimum adım sayısını belirtir. num_train_epochs yerine kullanıldığında, eğitim belirtilen adım sayısına ulaştığında durur.
        learning_rate = 2e-4, # Öğrenme hızını belirtir.
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1, # Her kaç adımda bir log kaydı yapılacağını belirtir.
        optim = "adamw_8bit", # Kullanılacak optimizasyon algoritmasını belirtir.
        weight_decay = 0.01, # Ağırlık azaltma değerini belirtir, bu da aşırı uyumu önlemeye yardımcı olur.
        lr_scheduler_type = "linear", # Öğrenme hızı zamanla nasıl değişeceğini belirten zamanlayıcı tipini ifade eder.
        seed = 3407,
        #do_eval=True,
        #eval_steps=100,
        output_dir = "outputs", # Eğitim sonuçlarının kaydedileceği dizini belirtir.
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/797739 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 797,739 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 2,000
 "-____-"     Number of trainable parameters = 50,003,968
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.623400
2,1.544300
3,1.427400
4,1.502700
5,1.494400
6,1.194800
7,1.355900
8,1.161300
9,1.190200
10,1.069100


# Tokenizer İşlemleri


In [9]:
# Dil modelleri, doğrudan ham metin üzerinde çalışamazlar. Bunun yerine, metni anlamlı bir şekilde işleyebilmek için metinleri öncelikle daha küçük birimlere ve sayısal değerlere ayırmaları gerekir.
# Bu işlem tokenizer tarafından yapılır ve "tokenization" olarak adlandırılır.
# Tokenization işleminin birkaç önemli nedeni vardır:
# 1. Standart Format: Modelin, eğitim sürecinde öğrendiği token formatı ile uyumlu girdiler alması gerekir. Eğitim sırasında model, belirli token dizilimlerini öğrenir ve bu dizilimlere göre tahminlerde bulunur.
# 2. Verimlilik: Tokenlar, dilin yapısal öğelerini temsil eder. Model, bu yapısal öğeleri kullanarak metni daha hızlı ve etkin bir şekilde işleyebilir. Örneğin, bazı yaygın kelimeler veya ifadeler tek bir token olarak temsil edilebilir, bu da modelin daha az işlem yaparak daha hızlı sonuçlar vermesini sağlar.
# 3. Anlamı Yakalama: Dilin karmaşıklığını ve anlamını daha iyi yakalamak için bazen kelimelerin kökleri veya morfolojik varyantları gibi daha derin dilbilimsel öğeler token olarak kullanılır. Bu, modelin dilin nüanslarını ve anlam katmanlarını daha iyi anlamasına olanak tanır.
# 4. Bellek ve İşlem Kapasitesi: Eğer model her bir karakter üzerinden çalışsaydı, çok daha fazla bilgiyi işlemesi ve saklaması gerekecekti. Tokenization sayesinde, model yalnızca anlamlı tokenlere odaklanır, bu da hem bellek kullanımını hem de işlem yükünü azaltır.

# Mevcut dil modelini doğrudan çıktı üretmek için optimize eder. Bu, modelin işlem süresini kısaltır ve daha hızlı sonuç alınmasını sağlar.
FastLanguageModel.for_inference(model)

# Burada bir metin listesi, modelin anlayabileceği bir forma (token'lar haline) dönüştürülür. Bu işlem sırasında nixous_prompt.format(...) fonksiyonu kullanılarak bir format şablonundan yararlanılır.
# Bu format şablonu, özellikle bir talimat verilerek, girdi ve çıktı alanları boş bırakılarak doldurulmuştur.
inputs = tokenizer(
[
    nixous_prompt.format(
        # Aşağıya soru yazılacak.
        "How do I use components to display my dynamic navigation bar using laravel?",
        "", # input - Instructiona bana vereceğim sayıları çarpar mısın yazmamız durumunda inputa 2x2 verebiliriz.
        "", # output - Boş bırakılmalıdır.
    )
], return_tensors = "pt").to("cuda") # return_tensors = "pt": Token'lar, PyTorch tensörlerine dönüştürülür.

# .to("cuda"): Elde edilen tensörler, CUDA uyumlu bir cihazda (genellikle bir GPU) işlenmek üzere gönderilir. Bu, işlemlerin çok daha hızlı yapılmasını sağlar.

# model.generate(...): Modelin, verilen girdilere dayanarak yeni metin üretmesi sağlanır.
# **inputs: Tokenizer tarafından oluşturulan girdi tensörleri.
# max_new_tokens = 64: Modelin, en fazla 64 yeni token (kelime veya kelime parçaları) üretebileceğini belirtir.
# use_cache = True: Modelin, hesaplamaları daha verimli yapmak için önceki hesaplamaların sonuçlarını önbellekte tutmasına izin verir.
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

print("outputs:")
print(outputs)

print("input:")
print(inputs)
# Model tarafından üretilen çıktılar (token'lar), insanların anlayabileceği düz metinlere dönüştürülür. Bu işlem, tokenizer'ın batch_decode fonksiyonu ile gerçekleştirilir.
tokenizer.batch_decode(outputs)

outputs:
tensor([[     2,  33501,    603,    671,  14239,    674,  19306,    476,   6911,
         235269,  44600,    675,    671,   3772,    674,   6572,   4024,   4807,
         235265,  15615,    476,   3590,    674,  47258,  56382,    573,   3853,
         235265,    109,   6176,  36142, 235292,    108,   2299,    749,    590,
           1281,   8832,    577,   4953,    970,  12415,  18897,   2333,   2177,
           1978,  14529, 235336,    109,   6176,  11438, 235292,    110,   6176,
          10567, 235292,    108,    187,    199,   5856, 235248, 235274,    208,
            194,    139, 235322, 235263, 235313,   4912,    476,    888,   2116,
            575,    861,   3542,    578,   1503,    665,    685, 235248,    207,
           4849, 235265,  41157, 235265,   1556,    216,    727, 235263, 235313,
            139, 235322,   1710, 235313,    207, 235343,   4526, 235289,    601,
           1062,    627,   4849,  76307,   4223, 235289,    142, 235343,   4526,
         235289,   

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nHow do I use components to display my dynamic navigation bar using laravel?\n\n### Input:\n\n\n### Response:\n<h3><strong>Step 1</strong></h3>  <p>Create a new view in your project and name it as <code>nav.blade.php</code></p>  <pre><code>&lt;ul class="nav"&gt;     &lt;li class="nav-item"&gt;&lt;a']

# TextStreamer ile Sonucun Gösterilmesi


In [10]:
# Mevcut dil modelini doğrudan çıktı üretmek için optimize eder. Bu, modelin işlem süresini kısaltır ve daha hızlı sonuç alınmasını sağlar.
FastLanguageModel.for_inference(model) #2x faster

# inputs = tokenizer([...], return_tensors = "pt").to("cuda"): Metni, model tarafından işlenebilecek token'lara çevirir. Bu, modelin metni anlamasına ve üzerinde işlem yapmasına olanak tanır.
#
inputs = tokenizer(
[
    nixous_prompt.format(
        # Soru aşağıya yazılacak.
        "How do I use components to display my dynamic navigation bar using laravel?", # instruction
         "",# input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer

# TextStreamer'ın ana görevi, metin üretilirken de ekrana yazdırmak. Örnek chatGPT cevap yazarken harfler
# tek tek ekrana düşerken, gemini çıktı üretildikten sonra ekrana basar.
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
How do I use components to display my dynamic navigation bar using laravel?

### Input:


### Response:
<h3><strong>Step 1</strong></h3>  <p>Create a new view in your project and name it as <code>nav.blade.php</code></p>  <pre><code>&lt;ul class="nav"&gt;     &lt;li class="nav-item"&gt;&lt;a class="nav-link active" href="http://localhost:8000"&gt;Home&lt;/a&gt;&lt;/li&gt;     &lt;li class="nav-item"&gt;&lt;a class="nav-link" href="http://localhost:8000/about"&gt;About&lt;/a&gt;&lt;/li&gt;     &lt;li class="nav-item"&gt;&lt;a class="nav-link" href="http://localhost:8000/contact"&gt;Contact&lt;/a&gt;&lt;/li&gt; &lt;/ul&gt; </code></pre>  <h3><strong>Step 2</strong></h3>  <p>Create a new view in your project and name it as <code>app.blade.php</code></p>  <pre><code>&lt;html&gt;     &lt;head&gt;         &lt;titl

# Eğitilen Modelin Export Edilmesi


In [11]:
model.save_pretrained("nixous_model")
tokenizer.save_pretrained("nixous_model")

('nixous_model/tokenizer_config.json',
 'nixous_model/special_tokens_map.json',
 'nixous_model/tokenizer.model',
 'nixous_model/added_tokens.json',
 'nixous_model/tokenizer.json')

# Eğitilen Model Üzerinden Sonuç Yazdırma


In [12]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./nixous_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    nixous_prompt.format(
        "How do I use components to display my dynamic navigation bar using laravel?", # instruction
        "", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")


outputs = model.generate(**inputs, max_new_tokens=500, use_cache=True)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
response = response.split("### Response:\n")[1].strip()

print(response)

==((====))==  Unsloth: Fast Gemma patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
<h3><strong>Step 1</strong></h3>  <p>Create a new view in your project and name it as <code>nav.blade.php</code></p>  <pre><code>&lt;ul class="nav"&gt;     &lt;li class="nav-item"&gt;&lt;a class="nav-link active" href="http://localhost:8000"&gt;Home&lt;/a&gt;&lt;/li&gt;     &lt;li class="nav-item"&gt;&lt;a class="nav-link" href="http://localhost:8000/about"&gt;About&lt;/a&gt;&lt;/li&gt;     &lt;li class="nav-item"&gt;&lt;a class="nav-link" href="http://localhost:8000/contact"&gt;Contact&lt;/a&gt;&lt;/li&gt; &lt;/ul&gt; </code></pre>  <h3><strong>Step 2</strong></h3>  <p>Create a new view in your project and name it as <code>app.blade.php</code

# Dosyaların ziplenmesi ve çıktılarının alınması


In [13]:
!zip -r huggingface_tokenizers_cache.zip /content/huggingface_tokenizers_cache

  adding: content/huggingface_tokenizers_cache/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--gemma-7b-bnb-4bit/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--gemma-7b-bnb-4bit/.no_exist/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--gemma-7b-bnb-4bit/.no_exist/d5efa09c28a16ede99e9df6b4e720ad4ee2c6a2b/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--gemma-7b-bnb-4bit/.no_exist/d5efa09c28a16ede99e9df6b4e720ad4ee2c6a2b/added_tokens.json (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--gemma-7b-bnb-4bit/refs/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--gemma-7b-bnb-4bit/refs/main (deflated 3%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--gemma-7b-bnb-4bit/blobs/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--gemma-7b-bnb-4bit/blobs/c15eb04bc5ad609fb26533e8525302c5640a

In [14]:
!zip -r nixous_model.zip /content/nixous_model

  adding: content/nixous_model/ (stored 0%)
  adding: content/nixous_model/tokenizer.json (deflated 72%)
  adding: content/nixous_model/tokenizer_config.json (deflated 96%)
  adding: content/nixous_model/adapter_model.safetensors (deflated 7%)
  adding: content/nixous_model/tokenizer.model (deflated 51%)
  adding: content/nixous_model/special_tokens_map.json (deflated 76%)
  adding: content/nixous_model/adapter_config.json (deflated 54%)
  adding: content/nixous_model/README.md (deflated 66%)


In [15]:
!zip -r outputs.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/checkpoint-2000/ (stored 0%)
  adding: content/outputs/checkpoint-2000/tokenizer.json (deflated 72%)
  adding: content/outputs/checkpoint-2000/tokenizer_config.json (deflated 96%)
  adding: content/outputs/checkpoint-2000/adapter_model.safetensors (deflated 7%)
  adding: content/outputs/checkpoint-2000/trainer_state.json (deflated 81%)
  adding: content/outputs/checkpoint-2000/tokenizer.model (deflated 51%)
  adding: content/outputs/checkpoint-2000/scheduler.pt (deflated 56%)
  adding: content/outputs/checkpoint-2000/optimizer.pt (deflated 17%)
  adding: content/outputs/checkpoint-2000/special_tokens_map.json (deflated 76%)
  adding: content/outputs/checkpoint-2000/training_args.bin (deflated 51%)
  adding: content/outputs/checkpoint-2000/adapter_config.json (deflated 54%)
  adding: content/outputs/checkpoint-2000/README.md (deflated 66%)
  adding: content/outputs/checkpoint-2000/rng_state.pth (deflated 25%)
  adding: con

In [16]:
!zip -r sample_data.zip /content/sample_data

  adding: content/sample_data/ (stored 0%)
  adding: content/sample_data/anscombe.json (deflated 83%)
  adding: content/sample_data/README.md (deflated 42%)
  adding: content/sample_data/california_housing_test.csv (deflated 76%)
  adding: content/sample_data/mnist_test.csv (deflated 88%)
  adding: content/sample_data/mnist_train_small.csv (deflated 88%)
  adding: content/sample_data/california_housing_train.csv (deflated 79%)


In [17]:
!zip -r wandb.zip /content/wandb

  adding: content/wandb/ (stored 0%)
  adding: content/wandb/latest-run/ (stored 0%)
  adding: content/wandb/latest-run/logs/ (stored 0%)
  adding: content/wandb/latest-run/logs/debug.log (deflated 72%)
  adding: content/wandb/latest-run/logs/debug-internal.log (deflated 95%)
  adding: content/wandb/latest-run/files/ (stored 0%)
  adding: content/wandb/latest-run/files/wandb-summary.json (deflated 44%)
  adding: content/wandb/latest-run/files/output.log (deflated 85%)
  adding: content/wandb/latest-run/files/wandb-metadata.json (deflated 75%)
  adding: content/wandb/latest-run/files/requirements.txt (deflated 55%)
  adding: content/wandb/latest-run/files/config.yaml (deflated 75%)
  adding: content/wandb/latest-run/run-ztpmodcv.wandb (deflated 77%)
  adding: content/wandb/latest-run/tmp/ (stored 0%)
  adding: content/wandb/latest-run/tmp/code/ (stored 0%)
  adding: content/wandb/debug.log (deflated 72%)
  adding: content/wandb/debug-internal.log (deflated 95%)
  adding: content/wandb/r